```
Muris Saab / Subin An
ADS509 - Final Project
University of San Diego

Title:
Automated Topic Classification and Discovery in News Articles Using Text Mining Techniques

Short Description:
This project aims to explore the automated classification and topic discovery of news articles gathered from various categories using the NewsAPI. The primary goal is to build a classification model to categorize articles into predefined groups such as "Technology," "Sports," "Health," and "Business," followed by topic modeling to uncover latent themes. The project will compare how well the discovered topics match with the pre-existing categories.

Objectives:
To collect raw news article data via the NewsAPI.
To clean, tokenize, and normalize the data for analysis.
To build a classification model to categorize the articles based on predefined categories.
To build a topic model to discover underlying themes in the data.
To evaluate the accuracy of the classification model and compare the topics discovered through topic modeling with the pre-existing categories.
To provide insights into how well the discovered topics align with the original article categories.


Query the dataset via public API
Note: free subscription only allows limited number of articles (30 days)

In [2]:
import requests
import pandas as pd

url = 'https://newsapi.org/v2/everything?q=trump&from=2024-09-05&to=2024-10-01&apiKey=c6ee2f8af5044d3a9f8647ad5c5e985a'

response = requests.get(url)

data = response.json()

articles = data['articles']
    
df = pd.DataFrame(articles)

print(df.head()) 


                                source             author  \
0    {'id': None, 'name': '[Removed]'}               None   
1  {'id': None, 'name': 'Gizmodo.com'}        Lucas Ropek   
2     {'id': 'wired', 'name': 'Wired'}       Jason Parham   
3     {'id': 'wired', 'name': 'Wired'}  Alistair Charlton   
4     {'id': 'wired', 'name': 'Wired'}    Lily Hay Newman   

                                               title  \
0                                          [Removed]   
1  The Postmaster General Is Stoking Election Fea...   
2  The Shade Room Founder Is Ready to Dial Down t...   
3  Trump's $100,000 Watches Are the Most Tragic C...   
4  The US Could Finally Ban Inane Forced Password...   

                                         description  \
0                                          [Removed]   
1  Like four years ago, the Trump-nominated Trump...   
2  Angie Nwandu dishes on Diddy, Donald Trump, th...   
3  Former president Donald Trump wants you to buy...   
4  Plus: The US 

In [ ]:
Save to CSV

In [3]:
df.to_csv('news_articles.csv', index=False)
